<a href="https://colab.research.google.com/github/jashvidesai/ORF-Thesis/blob/main/GreedyInitialSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import heapq

In [42]:
def greedy_initial_solution(V, V_star, K, Q, distances, vehicle_speed, d, p, a, b, s, c, fixed_costs):
    """
    Greedy algorithm for VRP initial solution, incorporating a priority queue for customer selection.
    No split deliveries though - computationally expensive!
    """
    routes = {k: [0] for k in K}  # Start all vehicles at depot
    loads = {k: 0 for k in K}  # Initial loads
    times = {k: 0 for k in K}  # Start times
    served = {j: 0 for j in V_star}  # Tracks amount served per customer
    total_costs = {k: fixed_costs[k] for k in K}  # Tracks costs, starting with fixed costs
    deliveries = {k: {} for k in K}  # Tracks deliveries per vehicle per node
    pickups = {k: {} for k in K}  # Tracks pickups per vehicle per node`

    # runs while there is unfulfilled demand
    while sum(served.values()) < sum(d[j] for j in V_star):
        for k in K:
            heap = []  # min-heap for selecting the best customer
            current_node = routes[k][-1]  # Last visited node

            # Find feasible customers and add them to the heap
            for j in V_star:
                remaining_demand = d[j] - served[j]
                if remaining_demand <= 0:
                    continue  # skip fully served customers

                # Calculate new load and arrival time
                arrival_time = times[k] + s[current_node] + (distances[current_node, j] / vehicle_speed)
                if arrival_time > b[j]:  # Early exit if time window is violated
                    continue

                # NOT allowing for split deliveries rn bc with the code below, takes too long
                if loads[k] + d[j] - p[j] <= Q[k]:
                    delivery_amount = d[j]
                    pickup_amount = p[j]
                else:
                    continue
                '''
                # Check the carrying capacity constraints, allowing for split deliveries
                delivery_amount = min(Q[k] - loads[k], d[j] - served[j])  # Deliver only as much as possible
                load_after_delivery = loads[k] + delivery_amount
                pickup_amount = min(p[j], Q[k] - load_after_delivery)
                load_final = load_after_delivery + pickup_amount

                if load_final > Q[k]:
                    continue  # skip if still exceeding
                # Update
                loads[k] = load_final
                served[j] += delivery_amount
                '''
                # Push directly into the heap instead of storing in a list first
                heapq.heappush(heap, (c[current_node, j], j))  # (cost, customer)

            # Select the best customer with minimum cost
            if heap:
                _, best_customer = heapq.heappop(heap)
                #delivery_amount = min(Q[k] - loads[k], d[best_customer] - served[best_customer])
                delivery_amount = d[best_customer]
                pickup_amount = p[best_customer]
            else:
                routes[k].append(0)  # No feasible customer, return to depot
                continue

            # Update vehicle route, load, and time
            routes[k].append(best_customer)
            loads[k] += p[best_customer] - delivery_amount
            times[k] += s[current_node] + (distances[current_node, best_customer] / vehicle_speed)
            total_costs[k] += c[current_node, best_customer]  # Add travel cost

            served[best_customer] += delivery_amount

            # Store delivery amount per vehicle per node
            if best_customer not in deliveries[k]:
                deliveries[k][best_customer] = 0
            deliveries[k][best_customer] += delivery_amount

            # Store pickup amount per vehicle per node
            if best_customer not in pickups[k]:
                pickups[k][best_customer] = 0
            pickups[k][best_customer] += pickup_amount

    # Ensure all vehicles return to depot
    for k in K:
        if routes[k][-1] != 0:
            routes[k].append(0)

    return routes, total_costs, deliveries, pickups

In [43]:
# Example usage
V = range(7)  # nodes including depot, 0 is depot
V_star = range(1, 7)  # nodes excluding depot
K = range(3)
Q = [25, 18, 20]

np.random.seed(42)
distances = np.array([
    [0, 35, 18, 23, 27, 48, 19],
    [35, 0, 25, 37, 34, 38, 31],
    [18, 25, 0, 27, 25, 30, 24],
    [23, 37, 27, 0, 40, 35, 26],
    [27, 34, 25, 40, 0, 26, 17],
    [48, 38, 30, 35, 26, 0, 39],
    [19, 31, 24, 26, 17, 39, 0]
])

vehicle_speed = 60  # vehicle speed in km/h
t = distances / vehicle_speed  # travel times
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)
fixed_costs = [110, 100, 105]  # fixed costs for each vehicle

d = [0, 50, 35, 20, 25, 30, 25]  # demands for each node
p = [0, 30, 22.5, 30, 45, 40, 28]  # pickup for each node
a = [0, 5, 10, 20, 20, 5, 10]  # Earliest arrival times
b = [30, 25, 35, 30, 30, 15, 25]  # Latest departure times
s = [0, 10, 10, 7, 5, 5, 10]  # Service times

# Run the optimized greedy heuristic
initial_routes, total_costs, deliveries, pickups = greedy_initial_solution(V, V_star, K, Q, distances, vehicle_speed, d, p, a, b, s, c, fixed_costs)

# Print the initial routes and costs
for k, route in initial_routes.items():
    print(f"Vehicle {k} route: {' -> '.join(map(str, route))}, Total Cost: {total_costs[k]:.2f}")

# Print deliveries per vehicle per node
print("\nDeliveries per vehicle:")
for k, delivery_info in deliveries.items():
    for node, amount in delivery_info.items():
        print(f"Vehicle {k} delivered {amount:.2f} units to node {node}")

# Print pickups per vehicle per node
print("\nPickups per vehicle:")
for k, pickup_info in pickups.items():
    for node, amount in pickup_info.items():
        print(f"Vehicle {k} picked up {amount:.2f} units to node {node}")

Vehicle 0 route: 0 -> 2 -> 1 -> 0, Total Cost: 114.00
Vehicle 1 route: 0 -> 6 -> 4 -> 0, Total Cost: 103.35
Vehicle 2 route: 0 -> 3 -> 5 -> 0, Total Cost: 110.39

Deliveries per vehicle:
Vehicle 0 delivered 35.00 units to node 2
Vehicle 0 delivered 50.00 units to node 1
Vehicle 1 delivered 25.00 units to node 6
Vehicle 1 delivered 25.00 units to node 4
Vehicle 2 delivered 20.00 units to node 3
Vehicle 2 delivered 30.00 units to node 5

Pickups per vehicle:
Vehicle 0 picked up 22.50 units to node 2
Vehicle 0 picked up 30.00 units to node 1
Vehicle 1 picked up 28.00 units to node 6
Vehicle 1 picked up 45.00 units to node 4
Vehicle 2 picked up 30.00 units to node 3
Vehicle 2 picked up 40.00 units to node 5


In [44]:
V = range(15)
V_star = range(1, 15)
K = range(4)
Q = [40, 35, 30, 25]

np.random.seed(42)
distances = np.array([
    [  0,  12,  22,  30,  25,  18,  20,  45,  28,  32,  27,  40,  35,  50,  38],
    [ 12,   0,  15,  27,  22,  20,  17,  40,  30,  25,  20,  37,  32,  45,  40],
    [ 22,  15,   0,  18,  25,  22,  19,  35,  40,  38,  30,  45,  37,  50,  42],
    [ 30,  27,  18,   0,  22,  20,  18,  32,  45,  37,  35,  42,  38,  50,  47],
    [ 25,  22,  25,  22,   0,  18,  20,  30,  35,  28,  25,  40,  32,  47,  38],
    [ 18,  20,  22,  20,  18,   0,  12,  27,  30,  22,  18,  35,  28,  42,  35],
    [ 20,  17,  19,  18,  20,  12,   0,  25,  32,  20,  22,  38,  27,  45,  38],
    [ 45,  40,  35,  32,  30,  27,  25,   0,  40,  37,  35,  42,  38,  50,  47],
    [ 28,  30,  40,  45,  35,  30,  32,  40,   0,  15,  20,  28,  25,  35,  30],
    [ 32,  25,  38,  37,  28,  22,  20,  37,  15,   0,  12,  30,  20,  42,  35],
    [ 27,  20,  30,  35,  25,  18,  22,  35,  20,  12,   0,  25,  15,  37,  28],
    [ 40,  37,  45,  42,  40,  35,  38,  42,  28,  30,  25,   0,  18,  28,  25],
    [ 35,  32,  37,  38,  32,  28,  27,  38,  25,  20,  15,  18,   0,  22,  18],
    [ 50,  45,  50,  50,  47,  42,  45,  50,  35,  42,  37,  28,  22,   0,  12],
    [ 38,  40,  42,  47,  38,  35,  38,  47,  30,  35,  28,  25,  18,  12,   0]
])

vehicle_speed = 60  # vehicle speed in km/h
t = distances / vehicle_speed  # travel times
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)
fixed_costs = [120, 115, 110, 100]

d = [0, 15, 10, 12, 8,  14, 9,  11, 10, 7,  12, 14, 13, 6,  9]
p = [0,  5,  3,  6, 4,   7, 2,   4,  6, 3,   5,  8,  7, 2,  3]
a = [0,  5,  10,  8,  12,  15,  18,  20,  10,  15,  18,  10,  12,  15,  18]
b = [60, 40, 35,  45, 30,  50,  55,  48,  60,  45,  50,  55,  65,  70,  75]
s = [0,  5,  5,  6, 4,  5,  6,  5,  6, 4,  5,  6,  7, 3,  5]

# Run the optimized greedy heuristic
initial_routes, total_costs, deliveries, pickups = greedy_initial_solution(V, V_star, K, Q, distances, vehicle_speed, d, p, a, b, s, c, fixed_costs)

# Print the initial routes and costs
for k, route in initial_routes.items():
    print(f"Vehicle {k} route: {' -> '.join(map(str, route))}, Total Cost: {total_costs[k]:.2f}")

# Print deliveries per vehicle per node
print("\nDeliveries per vehicle:")
for k, delivery_info in deliveries.items():
    for node, amount in delivery_info.items():
        print(f"Vehicle {k} delivered {amount:.2f} units to node {node}")

# Print pickups per vehicle per node
print("\nPickups per vehicle:")
for k, pickup_info in pickups.items():
    for node, amount in pickup_info.items():
        print(f"Vehicle {k} picked up {amount:.2f} units to node {node}")

Vehicle 0 route: 0 -> 1 -> 10 -> 9 -> 14 -> 0, Total Cost: 127.35
Vehicle 1 route: 0 -> 5 -> 4 -> 12 -> 13 -> 0, Total Cost: 123.37
Vehicle 2 route: 0 -> 6 -> 3 -> 11 -> 0, Total Cost: 117.44
Vehicle 3 route: 0 -> 2 -> 7 -> 8 -> 0, Total Cost: 109.02

Deliveries per vehicle:
Vehicle 0 delivered 15.00 units to node 1
Vehicle 0 delivered 12.00 units to node 10
Vehicle 0 delivered 7.00 units to node 9
Vehicle 0 delivered 9.00 units to node 14
Vehicle 1 delivered 14.00 units to node 5
Vehicle 1 delivered 8.00 units to node 4
Vehicle 1 delivered 13.00 units to node 12
Vehicle 1 delivered 6.00 units to node 13
Vehicle 2 delivered 9.00 units to node 6
Vehicle 2 delivered 12.00 units to node 3
Vehicle 2 delivered 14.00 units to node 11
Vehicle 3 delivered 10.00 units to node 2
Vehicle 3 delivered 11.00 units to node 7
Vehicle 3 delivered 10.00 units to node 8

Pickups per vehicle:
Vehicle 0 picked up 5.00 units to node 1
Vehicle 0 picked up 5.00 units to node 10
Vehicle 0 picked up 3.00 units 